<a href="https://colab.research.google.com/github/MuPeres/MotorFlawPrediction/blob/main/CD_phishing_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive #importar do drive

#CLASSIFICADORES
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [ ]:
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/MyDrive/archiveCD/CD_P1/phishing.csv')
df_train.head()

In [ ]:
binarize = ['UsingIP','ShortURL','Symbol@','Redirecting//','PrefixS9uffix-','DomainRegLen','Favicon','NonStdPort','HTTPSDomainURL','RequestURL','InfoEmail','AbnormalURL', 'StatusBarCust', 'DisableRightClick', 'UsingPopupWindow', 'IframeRedirection',  'AgeofDomain', 'DNSRecording', 'PageRank', 'GoogleIndex', 'class']
for classe in binarize:
    df_train[classe] = [0 if x== -1 else 1 for x in df_train[classe]]#-1=0, 1=1

adapt = ['LongURL','SubDomains','HTTPS','AnchorURL','LinksInScriptTags','WebsiteTraffic','LinksPointingToPage']
for tipo in adapt:
    df_train[tipo] = df_train[tipo].map({-1: 0, 0: 1, 1: 2})

In [ ]:
to_drop = ['Index']
df_train.drop(to_drop, axis=1, inplace=True)
X = df_train.drop(['class'], axis=1)
y = df_train['class']

In [ ]:
y.value_counts() #Balanceado

#Aplicando o classificador KNN para o Kfold com 10 splits

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
fold_knn = StratifiedKFold(n_splits=10)
accuracy_knn = np.array([])
roc_scores_knn= []

for i, (train, test) in enumerate(fold_knn.split(X, y), 1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i*12)
    knn.fit(X.iloc[train], y.iloc[train])
    y_pred = knn.predict(X.iloc[test])
    accuracy = accuracy_score(y[test], y_pred)

    # Calcula os Scores ROC AUC
    y_pred_proba = knn.predict(X.iloc[test])
    roc_score = roc_auc_score(y[test], y_pred_proba)
    roc_scores_knn.append(roc_score)

    mc=confusion_matrix(y[test], y_pred)

    print(f'Matriz confusão: \n {mc}')
    print(f"\n ROC AUC Score: {roc_score*100:,.2f}%")
    print("Accuracy KNN : %.2f%% \n" % (accuracy *100.0))
    accuracy_knn = np.append(accuracy_knn, accuracy)

accuracy_media = np.mean(accuracy_knn)
desvio_padrao = np.std(accuracy_knn)

avg_roc_score = sum(roc_scores_knn) / len(roc_scores_knn)
roc_std = np.std(roc_scores_knn)


print(f'\nAcurácia média KNN: {accuracy_media*100:,.2f}%\nDesvio padrão: {desvio_padrao*100:,.2f}%')

print(f"\n ROC AUC Score médio: {avg_roc_score*100:,.2f}%")
print(f"Desvio padrão do Score ROC AUC: {roc_std*100:,.2f}%")


#Normalizando

In [ ]:
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

#Extração de variáveis

In [ ]:
def remove_collinear_features(x, y, threshold):
    corr_matrix = x.corr().abs() #calcula a correlação absoluta (já que -1 é tão importante quanto 1) entre as colunas.
    target_corr = abs(x.corrwith(y)) #calcula a correlação absoluta de cada coluna com a classe target.

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) #analisa somente o triângulo acima da diagonal principal para evitar redundância nas iterações.

    drop_cols = []
    for column in upper.columns:
        correlated_cols = upper.index[upper[column] > threshold].tolist()
        if correlated_cols:
            least_corr_col = min(correlated_cols, key=lambda col: target_corr[col]) # de duas features de alta correlação, seleciona a feature que tem menor correlação com a classe target para ser excluída.
            drop_cols.append(least_corr_col) #guarda o nome das features a serem retiradas

    x = x.drop(columns=drop_cols) #retira as features listadas.
    return x

In [ ]:
rc = remove_collinear_features ((X),y, 0.9) #Considerando acima de 0.9 como correlação direta/inversa (valor absoluto).
rc

De forma visual



In [ ]:
'''
import seaborn as sns
import matplotlib.pyplot as plt


g = sns.clustermap(df_norm.corr(),
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8}
)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=25);
'''
corr = df_norm.corr()
styled_corr = corr.style.background_gradient(cmap='coolwarm')
styled_corr
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')
 .format(precision=3))


#Seleção de atributos

In [ ]:
def cor_selection(X, y, num_feat):
  cor_list = []
  feature_name = X.columns.tolist()
  for i in feature_name:
    cor = np.corrcoef(X[i], y)[0,1]
    cor_list.append(cor)

  column_count = len(X.columns)
  cor_feature = X.columns[np.argsort(np.abs(cor_list))][(column_count-num_feat):].tolist()
  return cor_feature

In [ ]:
feat_select = cor_selection(rc, y, 50)   #Lista a quantidade de atributos requisitada quanto a sua correlação com a classe Target, de maneira crescente
feat_select

In [ ]:
X_selected = rc[feat_select]
X_selected

# RandomForestClassifier

In [ ]:
trained_models = []

for i in range(10):

  X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_selected, y, test_size=0.3, random_state= i* 12)

  rf = RandomForestClassifier(n_estimators=100, criterion='entropy' , random_state= i * 12)  # Usando os estados testados anteriormente
  rf.fit(X_train_rf, y_train_rf)

  trained_models.append(rf)
  print(f"Random Forest {i+1} treinado com um random state igual a {i*12}.")

In [ ]:
accuracies = []
roc_scores = []
for i, model in enumerate(trained_models):

    y_pred = model.predict(X_test_rf)

    # Calcula as acurácias
    accuracy = accuracy_score(y_test_rf, y_pred)
    accuracies.append(accuracy)

    # Calcula os Scores ROC AUC
    y_pred_proba = model.predict_proba(X_test_rf)
    roc_score = roc_auc_score(y_test_rf, y_pred_proba[:,1])
    roc_scores.append(roc_score)

    mc2=confusion_matrix(y_test_rf, y_pred)

    print(f'Matriz confusão: \n {mc2}')
    print(f"Treino do RandomForest utilizado {i+1}:")
    print(f"Acurácia: {accuracy*100:,.2f}%")
    print(f"ROC AUC Score: {roc_score*100:,.2f}%\n")


avg_accuracy = sum(accuracies) / len(accuracies)
avg_roc_score = sum(roc_scores) / len(roc_scores)

accuracy_std = np.std(accuracies)
roc_std = np.std(roc_scores)


print(f"Acurácia média: {avg_accuracy*100:,.2f}%")
print(f"Desvio padrão da acurácia: {accuracy_std*100:,.2f}%")
print()
print(f"ROC AUC Score médio: {avg_roc_score*100:,.2f}%")
print(f"Desvio padrão do Score ROC AUC: {roc_std*100:,.2f}%")

Análise dos resultados

In [ ]:
from scipy.stats import wilcoxon

alpha = 0.05
_, p = wilcoxon(accuracy_knn, accuracies)
print(p)
if p > alpha:
  print('Mesma distribuição (Não rejeita H0)')
else:
  print('Distribuições diferentes (Rejeita H0)')

In [ ]:
import seaborn as sns

re1 = pd.DataFrame(accuracies)
re1['classifier'] = 'C1'

re2 = pd.DataFrame(accuracy_knn)
re2['classifier'] = 'C2'

result_clf = pd.concat([re1, re2], axis=0, ignore_index=True)
result_clf.columns = ['acc', 'classifier']
result_clf

sns.displot(result_clf, x='acc',kind="kde", fill=True, hue="classifier")

Analisando o gráfico de acurácia apresentado acima, é correto afirmar que o primeiro cenário (C1) possui menor dispersão quando comparado ao segundo cenário (C2), isso porque C1 possui um desvio padrão de apenas 0,45%, enquanto C2 possui 1,28% de desvio padrão. Além disso, é importante ressaltar que apesar dos desvios padrões obtidos por cada cenário, ambos apresentaram uma excelente capacidade em distinguir entre as classes positiva e negativa em um problema de classificação binária, o indicativo dessa característica é o ROC AUC Score que para C1 foi de 99,67% e para C2 95,28%, comprovando que ambos demonstraram precisão e obtiveram poucos erros.